In [92]:
import pandas as pd
import numpy as np
import math
import re
from datetime import datetime
from sklearn.preprocessing import *


import tensorflow as tf
from sklearn.model_selection import train_test_split

In [93]:
# DATAFRAME READ
raw_data = pd.read_csv('data_6.csv', header=0) # 1000 ROWS
# Clean full empty rows
raw_data = raw_data.dropna(how='all')
# -------------------------------------

In [94]:
for c in raw_data.columns:
    #if not c in ['Bedrooms', 'Bathrooms', 'SquareFootageHouse', 'Age', 'Location']:
    print(f"{c} -> {set(raw_data[c])}")

Bedrooms -> {1.0, 2.0, 3.0, 4.0, 5.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan}
Bathrooms -> {1.0, 2.0, 3.0, 4.0, 5.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan

## Data Preprocessing

In [95]:
preprocessed_data = raw_data.copy()

In [96]:
# RENAME VALUES

# Rename values from Location column Suburbann to Suburban
preprocessed_data.loc[raw_data['Location'] == 'Suburbann', 'Location'] = 'Suburban' # faster method

# Rename values from HeatingType column Oil Heating to Oil
preprocessed_data.loc[raw_data['HeatingType'] == 'Oil Heating', 'HeatingType'] = 'Oil'

# Rename values from HeatingType column Electricity Heating to Electric
preprocessed_data.loc[raw_data['HeatingType'] == 'Electricity', 'HeatingType'] = 'Electric'

# --------------------------------------------------------------------------

In [97]:
# DROP USELESS COLUMNS
preprocessed_data.drop(columns=['PreviousOwnerName', 'WindowModelNames', 'HouseColor'], inplace=True)

In [98]:
# DEAL WITH MISSING/ANOMALY VALUES (Age, Location, SquareFootageHouse)

# Drop negative values from Age
preprocessed_data = preprocessed_data.loc[(preprocessed_data['Age'] >= 0) | (preprocessed_data['Age'].isna())] # 14 rows less

# Drop rows where we have NAN values from Location and SquareFootageHouse rows
preprocessed_data = preprocessed_data.dropna(subset=['Location', 'SquareFootageHouse']) # 10 rows less
preprocessed_data = preprocessed_data[preprocessed_data['SquareFootageHouse'] > 0] # 4 rows less

# --------------------------------------------------------------------------

In [99]:
# MISSING VALUES - BEDROOM AND BATHROOM
# AVERAGE BASED ON SQUARE FOOTAGE HOUSE

min_sqr = preprocessed_data['SquareFootageHouse'].min()
max_sqr = preprocessed_data['SquareFootageHouse'].max()

bin_edges = np.arange(min_sqr,max_sqr+100, 100) #Making intervals from SquareFootageHouse with 100 as value.
preprocessed_data['SqrFHouse'] = pd.cut(preprocessed_data['SquareFootageHouse'], bins=bin_edges)


#Calculating the average of the bathrooms and bedrooms between this interval.

avg_bedrooms = preprocessed_data.groupby('SqrFHouse')['Bedrooms'].mean().apply(lambda x: math.floor(x) if pd.notnull(x) else x)
avg_bathrooms = preprocessed_data.groupby('SqrFHouse')['Bathrooms'].mean().apply(lambda x: math.floor(x) if pd.notnull(x) else x)

def fill_bedroom(row):
    if np.isnan(row['Bedrooms']):
        return avg_bedrooms.loc[row['SqrFHouse']]
    else:
        return row['Bedrooms']
    
    
def fill_bathroom(row):
    if np.isnan(row['Bathrooms']):
        return avg_bathrooms.loc[row['SqrFHouse']]
    else:
        return row['Bathrooms']



preprocessed_data['Bedrooms'] = preprocessed_data.apply(fill_bedroom, axis=1)
preprocessed_data['Bathrooms'] = preprocessed_data.apply(fill_bathroom, axis=1)

#Changing the rows with nan value to 0, because there are houses with no such features.
preprocessed_data['Bedrooms'] = preprocessed_data['Bedrooms'].fillna(0)
preprocessed_data['Bathrooms'] = preprocessed_data['Bathrooms'].fillna(0)


In [100]:
## Fill NaN and anomaly values

# With constant

#preprocessed_data['Bedrooms'] = preprocessed_data['Bedrooms'].fillna(0)
#preprocessed_data['Bathrooms'] = preprocessed_data['Bathrooms'].fillna(0)
preprocessed_data['Age'] = preprocessed_data['Age'].fillna(0)
preprocessed_data['PoolQuality'] = preprocessed_data['PoolQuality'].fillna('None')
preprocessed_data['HasPhotovoltaics'] = preprocessed_data['HasPhotovoltaics'].fillna(False)

In [101]:
# CREATING/MODIFYING NEW FEATURES
# -------------------------------------

# DaysSinceFromSale creation = (date of each house - 2024-06-23)
# -------------------------------------

def check_date_format(array):
    regex_pattern = r'^\d{4}-\d{2}-\d{2}$' # yyyy-mm-dd
    for value in array:
        if not re.match(regex_pattern, str(value)):
            return False
    return True

print(f"All dates from DateSinceForSale share the same format? {check_date_format(preprocessed_data['DateSinceForSale'])}")

preprocessed_data['DateSinceForSale'] = pd.to_datetime(preprocessed_data['DateSinceForSale'])

# Using the fixed date of 2024-06-23
fixed_date = datetime.strptime('2024-06-23', '%Y-%m-%d')

preprocessed_data['DaysSinceForSale'] = (fixed_date - preprocessed_data['DateSinceForSale']).dt.days

# Check for missing values
print(f"Missing values on DaysSinceForSale: {preprocessed_data['DaysSinceForSale'].isnull().sum()}")
preprocessed_data['DaysSinceForSale'].fillna(preprocessed_data['DaysSinceForSale'].mean(), inplace=True)

# HasPool creation
# -------------------------------------
preprocessed_data['HasPool'] = preprocessed_data['PoolQuality'].apply(lambda x: x != 'None')

# AverageQuality based on KitchensQuality, BathroomsQuality, BedroomsQuality, LivingRoomsQuality

quality_mapping = {
    'None' : -1,
    'Poor': 0,
    'Good': 1,
    'Excellent': 2
}
# Apply the mapping to each quality column
preprocessed_data['KitchensQuality'] = preprocessed_data['KitchensQuality'].map(quality_mapping)
preprocessed_data['BathroomsQuality'] = preprocessed_data['BathroomsQuality'].map(quality_mapping)
preprocessed_data['BedroomsQuality'] = preprocessed_data['BedroomsQuality'].map(quality_mapping)
preprocessed_data['LivingRoomsQuality'] = preprocessed_data['LivingRoomsQuality'].map(quality_mapping)

preprocessed_data['AverageQuality'] = preprocessed_data[['KitchensQuality', 
                                                         'BathroomsQuality', 
                                                         'BedroomsQuality', 
                                                         'LivingRoomsQuality']].mean(axis=1)

All dates from DateSinceForSale share the same format? True
Missing values on DaysSinceForSale: 0


In [102]:
# DROP AUXILIARY COLUMNS
preprocessed_data.drop(columns=['SqrFHouse', 'DateSinceForSale'], inplace=True)
# -------------------------------------

In [103]:
# CAST TO NUMERICAL
preprocessed_data['PoolQuality'] = preprocessed_data['PoolQuality'].map(quality_mapping)

location_mapping = {
    'Rural': 0,
    'Suburban': 1,
    'Urban': 2
}
preprocessed_data['Location'] = preprocessed_data['Location'].map(location_mapping)

heat_mapping = {
    'Oil' : 0,
    'Electric' : 1,
    'Gas' : 2
}

preprocessed_data['HeatingType'] = preprocessed_data['HeatingType'].map(heat_mapping)

preprocessed_data['HasPhotovoltaics'] = preprocessed_data['HasPhotovoltaics'].astype(int)
preprocessed_data['HasFiberglass'] = preprocessed_data['HasFiberglass'].astype(int)
preprocessed_data['IsFurnished'] = preprocessed_data['IsFurnished'].astype(int)
preprocessed_data['HasFireplace'] = preprocessed_data['HasFireplace'].astype(int)
preprocessed_data['HasPool'] = preprocessed_data['HasPool'].astype(int)
preprocessed_data['HeatingCosts'] = preprocessed_data['HeatingCosts'].astype(float)
preprocessed_data['SquareFootageHouse'] = preprocessed_data['SquareFootageHouse'].astype(float)
preprocessed_data['Price'] = preprocessed_data['Price'].astype(float)

In [104]:
# PREDICTION MODEL FOR HEATINGCOSTS ESTIMATION

# TODO: intentar one-hot enconding

# Preprocessing
HEATINGCOSTS_SUBSET = ['SquareFootageHouse', 'Location', 'HeatingType', 'HasPhotovoltaics', 'HasFiberglass', 'HasFireplace', 'HeatingCosts']
df = preprocessed_data[HEATINGCOSTS_SUBSET].copy()
df = df[df['HeatingCosts'] >= 0].dropna(subset=['HeatingCosts'])

# Splitting the data
X = df.drop('HeatingCosts', axis=1).values
y = df['HeatingCosts'].values

print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Define the model
heatingcosts_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

heatingcosts_model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())

# Train the model

# MinMax normalization
scaler_X = MinMaxScaler() # For later denormalization
scaler_y = MinMaxScaler()
X_train_norm = scaler_X.fit_transform(X_train)
X_test_norm = scaler_X.transform(X_test)
y_train_norm = scaler_y.fit_transform(y_train.reshape(-1, 1))
y_test_norm = scaler_y.transform(y_test.reshape(-1, 1))

heatingcosts_model.fit(X_train_norm, y_train_norm, epochs=100, batch_size=16)

# Evaluate the model
loss = heatingcosts_model.evaluate(X_test_norm, y_test_norm)
print(f'Test Loss: {loss}')

# Predict missing rows

# Get dataframe with the HeatingCosts nan and negative values
df = preprocessed_data[HEATINGCOSTS_SUBSET][(preprocessed_data['HeatingCosts'].isna()) | (preprocessed_data['HeatingCosts'] <= 0)].drop('HeatingCosts', axis=1).copy()

X_predict_norm = scaler_X.transform(df.values)

heatingcosts_predicted = np.round(scaler_y.inverse_transform(heatingcosts_model.predict(X_predict_norm)), 2)

idx_to_predict = preprocessed_data[(preprocessed_data['HeatingCosts'].isna()) | (preprocessed_data['HeatingCosts'] <= 0)].index.tolist()

i = 0
for idx in idx_to_predict:
    preprocessed_data.loc[idx, 'HeatingCosts'] = heatingcosts_predicted[i]
    i += 1

(493, 6)
Epoch 1/100
25/25 [==============================] - 1s 2ms/step - loss: 0.1340
Epoch 2/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0439
Epoch 3/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0184
Epoch 4/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0132
Epoch 5/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0117
Epoch 6/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0112
Epoch 7/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0109
Epoch 8/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0107
Epoch 9/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0107
Epoch 10/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0106
Epoch 11/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0105
Epoch 12/100
25/25 [==============================] - 0s 2ms/step - loss: 0.0105
Epoch 13/100
25/25 [========

In [105]:
sum(preprocessed_data['HeatingCosts'].isna())

0

In [106]:
for c in preprocessed_data.columns:
    #if not c in ['Bedrooms', 'Bathrooms', 'SquareFootageHouse', 'Age', 'Location']:
    print(f"{c} -> {set(preprocessed_data[c])}")

Bedrooms -> {0.0, 1.0, 2.0, 3.0, 4.0, 5.0}
Bathrooms -> {0.0, 1.0, 2.0, 3.0, 4.0, 5.0}
SquareFootageHouse -> {19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 7336.0, 5824.0, 5869.0, 5374.0, 8457.0, 

In [107]:
preprocessed_data.shape

(972, 22)

In [108]:
# SAVE FILE
preprocessed_data.to_csv('preprocessed.csv', index=False, header=True, sep=',', encoding='utf-8')

In [109]:
preprocessed_data

,Bedrooms,Bathrooms,SquareFootageHouse,Location,Age,PoolQuality,HasPhotovoltaics,HeatingType,HasFiberglass,IsFurnished,...,BathroomsQuality,BedroomsQuality,LivingRoomsQuality,SquareFootageGarden,PreviousOwnerRating,HeatingCosts,Price,DaysSinceForSale,HasPool,AverageQuality
0,1.0,2.0,127.0,0,69.0,1,0,1,1,1,...,1,1,1,16.0,1.33,134.220001,346.57594,272,1,1.00
1,3.0,3.0,129.0,0,65.0,-1,1,0,0,0,...,1,2,0,16.0,1.52,42.369999,429.91007,640,0,0.75
2,1.0,2.0,99.0,2,43.0,2,0,1,1,1,...,1,2,0,46.0,2.49,59.400000,243.80271,784,1,1.00
3,2.0,2.0,126.0,0,35.0,-1,0,0,0,0,...,1,1,1,6.0,1.44,201.600000,378.22772,1291,0,1.00
4,1.0,3.0,101.0,0,54.0,-1,0,1,0,1,...,0,0,1,6.0,1.88,60.000000,249.67852,638,0,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2.0,2.0,147.0,1,20.0,-1,0,1,0,0,...,0,1,1,6.0,2.37,303.829987,112.18396,1554,0,1.00
996,1.0,2.0,120.0,1,42.0,-1,1,1,1,0,...,2,1,1,6.0,2.40,104.400002,111.55332,351,0,1.25
997,1.0,5.0,137.0,2,0.0,0,0,1,1,1,...,1,2,1,14.0,1.50,82.200000,262.43546,790,1,1.50
998,2.0,3.0,126.0,2,40.0,-1,0,2,1,1,...,1,2,0,6.0,2.16,274.029999,234.58247,1620,0,0.75
